In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle
import scipy.stats as stats
import os


from set_home_directory import get_project_root_homedir_in_sys_path
project_root, main_dir = get_project_root_homedir_in_sys_path("inter_areal_predictability")
if project_root is None:
    raise RuntimeError(f"Project root not found: ensure a folder named '{project_root}' exists in one of the sys.path entries.")
print("Project root found:", project_root)

os.chdir(project_root)
fig_dir = os.path.join(project_root, 'results/paper_figures/')

import sys
sys.path.insert(0,os.path.join(main_dir,'utils/'))
sys.path.insert(0,main_dir)
from utils.neuron_properties_functions import get_dates, extract_mouse_name
from utils.fig_2_functions import make_mouse_df, plot_three_neurons, plot_correlations, make_monkey_df, get_ext_fig_stars, get_x_positions, make_monkey_df_poisson
from utils.ridge_regression_functions import get_predictions_evars_parallel
import utils.mouse_data_functions as cs
from utils.stats_functions import get_comparison_test_stars
from utils.macaque_data_functions import get_resps
save_figs = False

## upload stats

In [ ]:
mouse_stats_path = os.path.join(project_root, 'results/fig_2',f'mouse_stats.pkl')
with open(mouse_stats_path, 'rb') as f:
	mouse_stats = pickle.load(f)


monkey_names = ['L','A', 'D']
all_monkey_stats = {}
for monkey_name in monkey_names:
    monkey_stats_path = os.path.join(project_root, 'results/fig_2',f'monkey_{monkey_name}_stats.pkl')
    with open(monkey_stats_path, 'rb') as f:
        all_monkey_stats[monkey_name] = pickle.load(f)

## mouse predictions

In [ ]:
#create dataframe for plotting
df_mouse_all = make_mouse_df(mouse_stats)
df_mouse = df_mouse_all[df_mouse_all.control_shuffle==False]

In [ ]:
dataset_type = 'ori32'
mouse='ori32_M170717_MP033_2017-08-17'
area = 'L23'

# get mouse area info and sort by ev values
df_mini_info = df_mouse[(df_mouse['SNR']>=2)&(df_mouse['Split-half r']>=0.8)&(df_mouse['Mouse']==mouse)&(df_mouse['Area']==area)].sort_values('EV', ascending=False)

# Extract a percentage of top and bottom neuron indices
number_of_neurons = int(0.1 * len(df_mini_info))

top_df_rows = df_mini_info.iloc[:number_of_neurons]
bottom_df_rows = df_mini_info.iloc[-number_of_neurons:]
median_df_row = df_mini_info.iloc[np.argwhere(np.isclose(df_mini_info['EV'].values, df_mini_info['EV'].median(), rtol=1e-03, atol=1e-03, equal_nan=False))[0][0]]


mt = cs.mt_retriever(project_root, dataset_type=dataset_type)
# Retrieve responses for different layers
resp_L1, resp_L23, resp_L2, resp_L3, resp_L4 = mt.retrieve_layer_activity('resp', mouse)

# Select random neurons from top and bottom indices
np.random.seed(19)
top10neuron_mouse = np.random.choice(top_df_rows['Cell Number'].values)
np.random.seed(19)
bottom10neuron_mouse = np.random.choice(bottom_df_rows['Cell Number'].values)

med_neuron_mouse = median_df_row['Cell Number']
evar_neurons_mouse = [top10neuron_mouse,med_neuron_mouse, bottom10neuron_mouse]


sample_size=500
# Prepare data for ridge regression prediction
layer_to_predict = resp_L23
alpha = mouse_stats['mouse_alphas'][sample_size][(extract_mouse_name(mouse))][area]

# Perform ridge regression prediction using custom function
sorted_y_preds, evars = get_predictions_evars_parallel(resp_L4, layer_to_predict, alpha=alpha, n_splits=10, frames_reduced=5)

In [ ]:
# Set up the layout for plotting Fig 2A
n_rows = 3
n_columns = 1
fig, axes = plt.subplots(n_rows, n_columns, figsize=(3.35,1.2))

# Define frame start and stop for plotting
frame_start=890 
frame_stop =990

# Call a custom plotting function to visualize data for three selected neurons
plot_three_neurons(frame_start, frame_stop, layer_to_predict, sorted_y_preds, evar_neurons_mouse, 'black', axes, animal='mouse', ylim=None, condition_type=None)
if save_figs ==True:
    plt.savefig(fig_dir +'fig2_mouse_sample_predictions.pdf',transparent=True,bbox_inches='tight' )


In [ ]:
# Fig 2C
df_ = pd.DataFrame({'actual':layer_to_predict[:,top10neuron_mouse], 'predicted':sorted_y_preds[:,top10neuron_mouse]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = '#007069'
plot_correlations(df_, color, r_position=(0.05,0.9), ax=ax, ev=evars[top10neuron_mouse])
ax.set_aspect('equal')
ax.axis('equal')
if save_figs ==True:
    plt.savefig(fig_dir +'fig2_mouse_top10_corr.pdf', transparent=True,bbox_inches='tight', dpi=100 )
plt.show()

df = pd.DataFrame({'actual':layer_to_predict[:,med_neuron_mouse], 'predicted':sorted_y_preds[:,med_neuron_mouse]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = 'lightseagreen'
plot_correlations(df, color, r_position=(0.05,0.9), ax=ax, ev=evars[med_neuron_mouse])
ax.set_aspect('equal')
ax.axis('equal')
ax.set(ylabel=None)
if save_figs ==True:
    plt.savefig(fig_dir +'fig2_mouse_med_corr.pdf',transparent=True,bbox_inches='tight', dpi=100)
plt.show()

df = pd.DataFrame({'actual':layer_to_predict[:,bottom10neuron_mouse], 'predicted':sorted_y_preds[:,bottom10neuron_mouse]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = '#73BFB7'
plot_correlations(df, color, r_position=(0.05,0.9), ax=ax, ev=evars[bottom10neuron_mouse])
ax.set_aspect('equal')
ax.axis('equal')
ax.set(ylabel=None)
if save_figs ==True:
    plt.savefig(fig_dir +'fig2_mouse_bottom10_corr.pdf',transparent=True, bbox_inches='tight', format='pdf')
plt.show()



In [ ]:
#Fig 2E

med_evar = evars[med_neuron_mouse]
top_evar = evars[top10neuron_mouse]
bottom_evar = evars[bottom10neuron_mouse]


fig, ax = plt.subplots(figsize=(3.2,1.3))
sns.histplot(df_mouse[(df_mouse['Activity Type']=='stimulus')&(df_mouse.Area=='L23')&(df_mouse['Split-half r']>0.8)&(df_mouse['SNR']>=2)]['EV'], color='#222222', ax=ax,
             linewidth=0.4, binwidth=0.025)

ax.axvline(top_evar, color='#007069', linestyle='--', linewidth=1, label='cell #1', alpha=0.7)
ax.axvline(med_evar, color='lightseagreen', linestyle='--', linewidth=1, label='cell #2', alpha=0.7)
ax.axvline(bottom_evar, color='#73BFB7', linestyle='--', linewidth=1, label='cell #3', alpha=0.7)


data = df_mouse_all[(df_mouse_all['Activity Type']=='stimulus')& (df_mouse_all.SNR > 2) & (df_mouse_all['Split-half r'] > 0.8) & (df_mouse_all.control_shuffle == True) & (df_mouse_all.Area == 'L23')]['EV']
per_25 = np.percentile(data.values, 25)
per_75 = np.percentile(data.values, 75)
ax.axvspan(per_25, per_75, alpha=0.3, color='blue', label='shuffle\ncontrol\nIQR',
        linewidth=0,
        )

ax.set_xlabel('EV fraction', fontsize=6, labelpad=1)
ax.set_ylabel('No. of L2/3 Neurons', fontsize=6, labelpad=1)


# Increase the tick label size for both x and y axes
ax.tick_params(axis='x', labelsize=6, width=0.5, length=2, pad=1)
ax.tick_params(axis='y', labelsize=6, width=0.5, length=2, pad=1)
legend = ax.legend(loc=(0.85,0.5), fontsize=6*0.8)
legend.get_frame().set_linewidth(0.2)
ax.set_xticks(ticks=np.arange(0,0.9,0.2))
ax.spines[:].set_linewidth(0.5)
sns.despine()
if save_figs is True:
        plt.savefig(fig_dir +'fig2_mouse_hist.pdf',transparent=True,bbox_inches='tight' )
plt.show()

### print mean and std for text

In [ ]:
print('stimulus predictions')
print(f"mean = {df_mouse[(df_mouse['Activity Type']=='stimulus')&(df_mouse.Area=='L23')&(df_mouse['Split-half r']>0.8)&(df_mouse['SNR']>=2)]['EV'].mean():.2f}")
print(f"std = {df_mouse[(df_mouse['Activity Type']=='stimulus')&(df_mouse.Area=='L23')&(df_mouse['Split-half r']>0.8)&(df_mouse['SNR']>=2)]['EV'].std():.2f}")

print('\nshuffled control performance')
print(f"mean = {df_mouse_all[(df_mouse_all['Activity Type']=='stimulus')& (df_mouse_all.SNR > 2) & (df_mouse_all['Split-half r'] > 0.8) & (df_mouse_all.control_shuffle == True) & (df_mouse_all.Area == 'L23')]['EV'].mean():.3f}")
print(f"std = {df_mouse_all[(df_mouse_all['Activity Type']=='stimulus')& (df_mouse_all.SNR > 2) & (df_mouse_all['Split-half r'] > 0.8) & (df_mouse_all.control_shuffle == True) & (df_mouse_all.Area == 'L23')]['EV'].std():.3f}")

#### supplemental

In [ ]:
#Fig Supp 1A
fig, ax = plt.subplots(figsize=(2.3,1.3))
sns.histplot(df_mouse[(df_mouse['Activity Type']=='stimulus')&(df_mouse.Area=='L4')&(df_mouse['Split-half r']>0.8)&(df_mouse['SNR']>=2)]['EV'], color='#222222', ax=ax,
        linewidth=0.4, binwidth=0.025)

data = df_mouse_all[(df_mouse_all['Activity Type']=='stimulus')& (df_mouse_all.SNR > 2) & (df_mouse_all['Split-half r'] > 0.8) & (df_mouse_all.control_shuffle == True) & (df_mouse_all.Area == 'L4')]['EV']
per_25 = np.percentile(data.values, 25)
per_75 = np.percentile(data.values, 75)
ax.axvspan(per_25, per_75, alpha=0.3, color='blue', label='shuffle\ncontrol\nIQR',
        linewidth=0)
mean_EV = df_mouse[(df_mouse['Activity Type']=='stimulus')&(df_mouse.Area=='L4')&(df_mouse['Split-half r']>0.8)&(df_mouse['SNR']>=2)]['EV'].mean()
ax.axvline(mean_EV, color='#EDAEAE', linestyle='--', linewidth=1, label=f'mean', alpha=0.7)
ax.set_xlabel('EV fraction', fontsize=6, labelpad=1)
ax.set_ylabel('No. of L4 Neurons', fontsize=6, labelpad=1)


# Increase the tick label size for both x and y axes
ax.tick_params(axis='x', labelsize=6, width=0.5, length=2, pad=1)
ax.tick_params(axis='y', labelsize=6, width=0.5, length=2, pad=1)
legend = ax.legend(loc=(0.7,0.5), fontsize=6*0.8)
legend.get_frame().set_linewidth(0.2)
ax.set_xticks(ticks=np.arange(0,0.9,0.2))
ax.spines[:].set_linewidth(0.5)
sns.despine()
if save_figs is True:
        plt.savefig(fig_dir +'fig2_supp_L4_mouse_hist.pdf',transparent=True,bbox_inches='tight' )
plt.show()

In [ ]:
## label visual responsive or non-responsive neurons
df_mouse = df_mouse.copy()  # Ensure we are working with a copy

df_mouse.loc[:,'Visual\nResponsiveness']=['responsive' if x > 0.8 and y > 2 else 'non-responsive' for x, y in zip(df_mouse['Split-half r'], df_mouse['SNR'])]
df_mouse.loc[:,'SNR > 2']=[True if y > 2 else False for _, y in zip(df_mouse['Split-half r'], df_mouse['SNR'])]
df_mouse.loc[:,'reliable']=[True if x > 0.8 else False for x, _ in zip(df_mouse['Split-half r'], df_mouse['SNR'])]
print(df_mouse.groupby(['Visual\nResponsiveness'])['EV'].agg('count'))


In [ ]:
# Fig Supp 1B
fig, ax = plt.subplots(figsize=(1.3,1.3))
plot = sns.histplot(df_mouse, x='EV', fill=False, hue='Visual\nResponsiveness', color='#222222', ax=ax,hue_order=['responsive','non-responsive'],
            linewidth=0.4, bins=20, palette=['purple', 'gray'], stat='percent')
ax.axvline(df_mouse[df_mouse['Visual\nResponsiveness']=='responsive']['EV'].mean(), color='purple', linestyle='--', linewidth=1, label='responsive\nmean EV', alpha=1)
ax.axvline(df_mouse[df_mouse['Visual\nResponsiveness']=='non-responsive']['EV'].mean(), color='gray', linestyle='--', linewidth=1, label='non-responsive\nmean EV', alpha=1)

ax.set_xlabel('EV fraction', fontsize=6, labelpad=1)
ax.set_ylabel(f'Percent of Neurons', fontsize=6, labelpad=1)
# Increase the tick label size for both x and y axes
ax.tick_params(axis='x', labelsize=6, width=0.5, length=2, pad=1)
ax.tick_params(axis='y', labelsize=6, width=0.5, length=2, pad=1)

ax.set_xticks(ticks=np.arange(0,0.9,0.2))
ax.spines[:].set_linewidth(0.5)
sns.despine()
legend = ax.get_legend()
handles1= legend.legend_handles
handles2, labels = ax.get_legend_handles_labels()
handles = handles1 + handles2
ax.legend(handles=handles, 
        labels=['responsive','non-responsive', 'responsive\nmean','non-responsive\nmean'], 
        fontsize=6*0.8,
        loc=(0.4,0.5),
        alignment='center')
ax.legend_.get_frame().set_linewidth(0.2)
if save_figs is True:
        plt.savefig(fig_dir +'fig2_supp_mouse_visual_nonvisual.pdf',transparent=True,bbox_inches='tight' )
plt.show()

In [ ]:
df_all = df_mouse.copy().reset_index()
df_all['cell type']='all'
df_snr = df_mouse[df_mouse['SNR > 2']==True].reset_index()
df_snr['cell type']='SNR > 2'
df_filt = df_mouse[df_mouse['Visual\nResponsiveness']=='responsive'].reset_index()
df_filt['cell type']='SNR > 2 &\nsplit-half r >0.8'
df_reli = df_mouse[df_mouse['reliable']==True].reset_index()
df_reli['cell type']='split-half r >0.8'
df_conditions_comparisons_mouse = pd.concat([df_all,df_reli,df_snr, df_filt]).reset_index()

In [ ]:
# show neuron counts across cell types and areas
print(df_conditions_comparisons_mouse.groupby(['Mouse Name','cell type','Area'])['EV'].agg('count').unstack())




In [ ]:
df_counts = df_conditions_comparisons_mouse.groupby(['Mouse Name','Area','cell type',])['EV'].agg('count').unstack()
df_counts['SNR > 2 (percent)'] = (df_counts['SNR > 2'] / df_counts['all']) * 100
df_counts['SNR > 2 &\nsplit-half r >0.8 (percent)'] = (df_counts['SNR > 2 &\nsplit-half r >0.8'] / df_counts['all']) * 100
df_counts['split-half r >0.8 (percent)'] = (df_counts['split-half r >0.8'] / df_counts['all']) * 100


In [ ]:
print(df_counts[['SNR > 2 (percent)','SNR > 2 &\nsplit-half r >0.8 (percent)','split-half r >0.8 (percent)']].groupby(['Area']).agg(['mean']))

In [ ]:
all_stars = get_ext_fig_stars(df_conditions_comparisons_mouse)

In [ ]:
# Fig Supp 1C
fig, ax = plt.subplots(figsize=(3.8,1.5))
violin = sns.violinplot(df_conditions_comparisons_mouse,  y='EV', x='Mouse Name', hue='cell type', ax=ax,
            palette=['lightgray','#ffb3ba','#bae1ff','#d0a0ec'],
            inner_kws={'box_width':2, 'marker':'_', 'markersize':3,
                            'markeredgewidth':0.75,'alpha':0.5},
            gap=-.3, cut=0,
            linewidth=0.2)
ax.set_xlabel(None)
ax.set_ylabel(f'EV fraction', fontsize=6, labelpad=1)

ax.tick_params(axis='x', labelsize=6, width=0.5, length=2, pad=1)
ax.tick_params(axis='y', labelsize=6, width=0.5, length=2, pad=1)
ax.legend(alignment='center',
title_fontsize= 6, fontsize=6, loc= (1,0.3),
borderpad=0.75)
ax.legend_.get_frame().set_linewidth(0.2)
ax.legend_.remove()
ax.spines[:].set_linewidth(0.5)
sns.despine()

x_positions = get_x_positions(violin)   
y_positions = [0.7,0.7,0.7,
            0.6,0.6,0.6,
            0.5,0.5,0.5,
            0.7,0.7,0.7,
            ]
for count, (x_pos, star, y_pos) in enumerate(zip(x_positions, all_stars,y_positions)):
    ax.text(x=x_pos-0.15, y=y_pos, s=star, 
            fontsize=5)
plt.savefig(fig_dir +'fig2_supp_mouse_EV_filter_comparisons.pdf',transparent=True,bbox_inches='tight' )
plt.show()

### Print table stats for mouse

In [ ]:
df_conditions_comparisons_mouse.dropna(subset='EV').groupby(['Area', 'cell type' ])['EV'].agg('count')

In [ ]:
df_conditions_comparisons_mouse.groupby(['Dataset Type', 'Mouse Name', 'Area', 'cell type' ])['EV'].agg('count')


## macaque L predictions

### plotting

In [ ]:
df_monkeys_all=pd.DataFrame()
monkey_name = 'L'
monkey_stats = all_monkey_stats[monkey_name]
df_monkey_all = make_monkey_df(monkey_stats)
df_monkey = df_monkey_all[df_monkey_all.control_shuffle==False]

df_monkey_all['Monkey Name'] = monkey_name
df_monkeys_all = pd.concat([df_monkeys_all, df_monkey_all])

df_monkey_V1_L = df_monkey[(df_monkey['Area']=='V1')&(df_monkey['Split-half r']>0.8)&(df_monkey['SNR']>2)].copy()
df_monkey_V1_L['Monkey Name']= 'L'

In [ ]:
condition_type= 'SNR'
date = get_dates(condition_type, monkey=monkey_name)[1]


# get mouse area info and sort by ev values
df_mini_info_monkey = df_monkey[(df_monkey['SNR']>=2)&(df_monkey['Split-half r']>=0.8)&(df_monkey['Date']==date)&(df_monkey['Area']=='V4')].sort_values('EV', ascending=False)

# Extract a percentage of top and bottom neuron indices
number_of_neurons = int(0.1 * len(df_mini_info_monkey))

top_df_rows_monkey = df_mini_info_monkey.iloc[:number_of_neurons]
bottom_df_rows_monkey = df_mini_info_monkey.iloc[-number_of_neurons:]
median_df_row_monkey= df_mini_info_monkey.iloc[np.argwhere(np.isclose(df_mini_info_monkey['EV'].values, df_mini_info_monkey['EV'].median(), rtol=1e-03, atol=1e-03, equal_nan=False))[0][0]]



# Select random neurons from top and bottom indices
np.random.seed(19)
top10neuron_monkey = np.random.choice(top_df_rows_monkey['Cell Number'].values)
np.random.seed(19)
bottom10neuron_monkey = np.random.choice(bottom_df_rows_monkey['Cell Number'].values)
med_neuron_monkey = median_df_row_monkey['Cell Number']

evar_neurons_monkey = [top10neuron_monkey,med_neuron_monkey, bottom10neuron_monkey]


resp_V4, resp_V1 =get_resps(condition_type=condition_type, date=date, w_size=25, stim_on=0, stim_off=400, monkey=monkey_name)
layer_to_predict = resp_V4
layer_to_use = resp_V1

frame_size = 500
alpha = all_monkey_stats[monkey_name]['monkey_alphas'][frame_size]['V4']
sorted_y_preds,evars = get_predictions_evars_parallel(layer_to_use, layer_to_predict, alpha=alpha, 
                                                        n_splits=10, frames_reduced=5)

In [ ]:
# Fig 2B
n_rows = 3
n_columns = 1
fig, axes = plt.subplots(n_rows, n_columns, figsize=(3.35,1.2))
frame_start=25
frame_stop =40
plot_three_neurons(frame_start, frame_stop, layer_to_predict, sorted_y_preds, evar_neurons_monkey, 'black', axes, animal='monkey', ylim=None, condition_type='SNR')
if save_figs is True:
    plt.savefig(fig_dir +'fig2_monkey_sample_predictions.pdf',transparent=True,bbox_inches='tight' )
plt.show()

In [ ]:
# Fig 2D
df_ = pd.DataFrame({'actual':layer_to_predict[:,top10neuron_monkey], 'predicted':sorted_y_preds[:,top10neuron_monkey]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = '#007069'
plot_correlations(df_, color, r_position=(0.05,0.9), ax=ax, ev=evars[top10neuron_monkey])
ax.set_aspect('equal')
ax.axis('equal')
if save_figs is True: 
    plt.savefig(fig_dir +'fig2_monkey_top10_corr.pdf', transparent=True,bbox_inches='tight', dpi=100 )
plt.show()

df = pd.DataFrame({'actual':layer_to_predict[:,med_neuron_monkey], 'predicted':sorted_y_preds[:,med_neuron_monkey]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = 'lightseagreen'
plot_correlations(df, color, r_position=(0.05,0.9), ax=ax, ev=evars[med_neuron_monkey])
ax.set_aspect('equal')
ax.axis('equal')
ax.set_ylabel(None)
if save_figs is True:
    plt.savefig(fig_dir +'fig2_monkey_med_corr.pdf',transparent=True,bbox_inches='tight' )
plt.show()

df = pd.DataFrame({'actual':layer_to_predict[:,bottom10neuron_monkey], 'predicted':sorted_y_preds[:,bottom10neuron_monkey]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = '#73BFB7'
plot_correlations(df, color, r_position=(0.05,0.9), ax=ax, ev=evars[bottom10neuron_monkey])
ax.set_aspect('equal')
ax.axis('equal')
ax.set_ylabel(None)
if save_figs is True:
    plt.savefig(fig_dir +'fig2_monkey_bottom10_corr.pdf',transparent=True,bbox_inches='tight' )
plt.show()

In [ ]:
#Fig 2F

med_evar = evars[med_neuron_monkey]
top_evar = evars[top10neuron_monkey]
bottom_evar = evars[bottom10neuron_monkey]


fig, ax = plt.subplots(figsize=(3.2,1.3))
sns.histplot(df_monkey[(df_monkey['Area']=='V4')&(df_monkey['Split-half r']>0.8)&(df_monkey['SNR']>2)]['EV'], 
                color='#222222', ax=ax, binwidth=0.025, linewidth=0.4)

ax.axvline(top_evar, color='#007069', linestyle='--', linewidth=1, label='site #1', alpha=0.7)
ax.axvline(med_evar, color='lightseagreen', linestyle='--', linewidth=1, label='site #2', alpha=0.7)
ax.axvline(bottom_evar, color='#73BFB7', linestyle='--', linewidth=1, label='site #3', alpha=0.7)


data = df_monkey_all[(df_monkey_all['Split-half r']>0.8)&(df_monkey_all['SNR']>2)&(df_monkey_all.control_shuffle==True)&(df_monkey_all.Area=='V4')]['EV']
per_25 = np.percentile(data.values, 25)
per_75 = np.percentile(data.values, 75)
ax.axvspan(per_25, per_75, alpha=0.3, color='blue', label='shuffle\ncontrol\nIQR',
        linewidth=0,
        )

ax.set_xlabel('EV fraction', fontsize=6, labelpad=1)
ax.set_ylabel('No. of V4 neural sites', fontsize=6, labelpad=1)

# Increase the tick label size for both x and y axes
ax.tick_params(axis='x', labelsize=6, width=0.5, length=2, pad=1)
ax.tick_params(axis='y', labelsize=6, width=0.5, length=2, pad=1)
legend = ax.legend(loc=(0.775,0.5), fontsize=6*0.8)
legend.get_frame().set_linewidth(0.2)
ax.set_xticks(ticks=np.arange(0,0.9,0.2))
ax.spines[:].set_linewidth(0.5)
sns.despine()
if save_figs is True:
        plt.savefig(fig_dir +'fig2_monkey_hist.pdf',transparent=True,bbox_inches='tight' )
plt.show()

### print mean and std for paper 

In [ ]:
print('stimulus predictions')
print(f"mean = {df_monkey[(df_monkey['Area']=='V4')&(df_monkey['Split-half r']>0.8)&(df_monkey['SNR']>2)]['EV'].mean():.2f}")
print(f"std = {df_monkey[(df_monkey['Area']=='V4')&(df_monkey['Split-half r']>0.8)&(df_monkey['SNR']>2)]['EV'].std():.2f}")

print('\nshuffled control performance')
print(f"mean = {df_monkey_all[(df_monkey_all['Split-half r']>0.8)&(df_monkey_all['SNR']>2)&(df_monkey_all.control_shuffle==True)&(df_monkey_all.Area=='V4')]['EV'].mean():.3f}")
print(f"std = {df_monkey_all[(df_monkey_all['Split-half r']>0.8)&(df_monkey_all['SNR']>2)&(df_monkey_all.control_shuffle==True)&(df_monkey_all.Area=='V4')]['EV'].std():.3f}")

## macaque A predictions

### plotting

In [ ]:
monkey_name = 'A'
monkey_stats = all_monkey_stats[monkey_name]
df_monkey_all = make_monkey_df(monkey_stats)
df_monkey = df_monkey_all[df_monkey_all.control_shuffle==False]
df_monkey_all['Monkey Name'] = monkey_name
df_monkeys_all = pd.concat([df_monkeys_all, df_monkey_all])
df_monkey_V1_A = df_monkey[(df_monkey['Area']=='V1')&(df_monkey['Split-half r']>0.8)&(df_monkey['SNR']>2)].copy()
df_monkey_V1_A['Monkey Name']= 'A'

In [ ]:
#Fig Supp 1D
n_rows = 3
n_columns = 1
fig, axes = plt.subplots(n_rows, n_columns, figsize=(3.35,1.2))
frame_start=25
frame_stop =40
plot_three_neurons(frame_start, frame_stop, layer_to_predict, sorted_y_preds, evar_neurons_monkey, 'black', axes, animal='monkey', ylim=None, condition_type='SNR')
if save_figs is True:
    plt.savefig(fig_dir + f'fig2_monkey_{monkey_name}_sample_predictions.pdf',transparent=True,bbox_inches='tight' )
plt.show()

In [ ]:
#Fig Supp 1F

df_ = pd.DataFrame({'actual':layer_to_predict[:,top10neuron_monkey], 'predicted':sorted_y_preds[:,top10neuron_monkey]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = '#007069'
plot_correlations(df_, color, r_position=(0.05,0.9), ax=ax, ev=evars[top10neuron_monkey])
ax.set_aspect('equal')
ax.axis('equal')
if save_figs is True: 
    plt.savefig(fig_dir + f'fig2_monkey_{monkey_name}_top10_corr.pdf', transparent=True,bbox_inches='tight', dpi=100 )
plt.show()

df = pd.DataFrame({'actual':layer_to_predict[:,med_neuron_monkey], 'predicted':sorted_y_preds[:,med_neuron_monkey]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = 'lightseagreen'
plot_correlations(df, color, r_position=(0.05,0.9), ax=ax, ev=evars[med_neuron_monkey])
ax.set_aspect('equal')
ax.axis('equal')
ax.set_ylabel(None)
if save_figs is True:
    plt.savefig(fig_dir + f'fig2_monkey_{monkey_name}_med_corr.pdf',transparent=True,bbox_inches='tight' )
plt.show()

df = pd.DataFrame({'actual':layer_to_predict[:,bottom10neuron_monkey], 'predicted':sorted_y_preds[:,bottom10neuron_monkey]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = '#73BFB7'
plot_correlations(df, color, r_position=(0.05,0.9), ax=ax, ev=evars[bottom10neuron_monkey])
ax.set_aspect('equal')
ax.axis('equal')
ax.set_ylabel(None)
if save_figs is True:
    plt.savefig(fig_dir + f'fig2_monkey_{monkey_name}_bottom10_corr.pdf',transparent=True,bbox_inches='tight' )
plt.show()

In [ ]:
#Fig Supp 1H

med_evar = evars[med_neuron_monkey]
top_evar = evars[top10neuron_monkey]
bottom_evar = evars[bottom10neuron_monkey]


fig, ax = plt.subplots(figsize=(1.3,1.3))
sns.histplot(df_monkey[(df_monkey['Area']=='V4')&(df_monkey['Split-half r']>0.8)&(df_monkey['SNR']>2)]['EV'], 
                color='#222222', ax=ax, binwidth=0.025, linewidth=0.4)

ax.axvline(top_evar, color='#007069', linestyle='--', linewidth=1, label='site #1', alpha=0.7)
ax.axvline(med_evar, color='lightseagreen', linestyle='--', linewidth=1, label='site #2', alpha=0.7)
ax.axvline(bottom_evar, color='#73BFB7', linestyle='--', linewidth=1, label='site #3', alpha=0.7)


data = df_monkey_all[(df_monkey_all['Split-half r']>0.8)&(df_monkey_all['SNR']>2)&(df_monkey_all.control_shuffle==True)&(df_monkey_all.Area=='V4')]['EV']
per_25 = np.percentile(data.values, 25)
per_75 = np.percentile(data.values, 75)
ax.axvspan(per_25, per_75, alpha=0.3, color='blue',
        linewidth=0,
        )

ax.set_xlabel('EV fraction', fontsize=6, labelpad=1)
ax.set_ylabel('No. of V4 neural sites', fontsize=6, labelpad=1)

# Increase the tick label size for both x and y axes
ax.tick_params(axis='x', labelsize=6, width=0.5, length=2, pad=1)
ax.tick_params(axis='y', labelsize=6, width=0.5, length=2, pad=1)
legend = ax.legend(loc=(0.775,0.5), fontsize=6*0.8)
legend.get_frame().set_linewidth(0.2)
ax.set_xticks(ticks=np.arange(0,0.8,0.2))
ax.spines[:].set_linewidth(0.5)
sns.despine()
if save_figs is True:
        plt.savefig(fig_dir + f'fig2_monkey_{monkey_name}_hist.pdf',transparent=True,bbox_inches='tight' )
plt.show()

## macaque D predictions

### plotting

In [ ]:

monkey_name = 'D'
monkey_stats = all_monkey_stats[monkey_name]
df_monkey_all = make_monkey_df(monkey_stats, dataset_types=['SNR'])
df_monkey = df_monkey_all[df_monkey_all.control_shuffle==False]
df_monkey_all['Monkey Name'] = monkey_name
df_monkeys_all = pd.concat([df_monkeys_all, df_monkey_all])
df_monkey_V1_D = df_monkey[(df_monkey['Area']=='V1')&(df_monkey['Split-half r']>0.6)&(df_monkey['SNR']>2)].copy()
df_monkey_V1_D['Monkey Name']= 'D'
all_df_V1 = pd.concat([df_monkey_V1_L, df_monkey_V1_A, df_monkey_V1_D])
reli_theshold = 0.6

In [ ]:

condition_type= 'SNR'
date = '260225'
reli_theshold = 0.6

# get mouse area info and sort by ev values
df_mini_info_monkey = df_monkey[(df_monkey['SNR']>=2)&(df_monkey['Split-half r']>=reli_theshold)&(df_monkey['Date']==date)&(df_monkey['Area']=='V4')].sort_values('EV', ascending=False)

# Extract a percentage of top and bottom neuron indices
number_of_neurons = int(0.1 * len(df_mini_info_monkey)) if  int(0.1 * len(df_mini_info_monkey)) > 0 else 1

top_df_rows_monkey = df_mini_info_monkey.iloc[:number_of_neurons]
bottom_df_rows_monkey = df_mini_info_monkey.iloc[-number_of_neurons:]
median_df_row_monkey= df_mini_info_monkey.iloc[np.argwhere(np.isclose(df_mini_info_monkey['EV'].values, df_mini_info_monkey['EV'].median(), rtol=1e-01, atol=1e-02, equal_nan=False))[0][0]]


# Select random neurons from top and bottom indices
np.random.seed(17)
top10neuron_monkey = np.random.choice(top_df_rows_monkey['Cell Number'].values)
np.random.seed(19)
bottom10neuron_monkey = np.random.choice(bottom_df_rows_monkey['Cell Number'].values)
med_neuron_monkey = median_df_row_monkey['Cell Number']

evar_neurons_monkey = [top10neuron_monkey,med_neuron_monkey, bottom10neuron_monkey]


resp_V4, resp_V1 =get_resps(condition_type=condition_type, date=date, w_size=25, stim_on=0, stim_off=400, monkey=monkey_name,
                            )
layer_to_predict = resp_V4
# subtract the mean response across all neurons to center the data
layer_to_predict = layer_to_predict - np.mean(layer_to_predict, axis=0)
layer_to_use = resp_V1

frame_size = 500
alpha = all_monkey_stats[monkey_name]['monkey_alphas'][frame_size]['V4']
sorted_y_preds,evars = get_predictions_evars_parallel(layer_to_use, layer_to_predict, alpha=alpha, 
                                                        n_splits=10, frames_reduced=5)

In [ ]:
#Fig Supp 1E
n_rows = 3
n_columns = 1
fig, axes = plt.subplots(n_rows, n_columns, figsize=(3.35,1.2))
frame_start=18
frame_stop =33
plot_three_neurons(frame_start, frame_stop, layer_to_predict, sorted_y_preds, evar_neurons_monkey, 'black', axes, animal='monkey', 
                ylim=None, condition_type='SNR')
if save_figs is True:
    plt.savefig(fig_dir + f'fig2_monkey_{monkey_name}_sample_predictions.pdf',transparent=True,bbox_inches='tight' )
plt.show()

In [ ]:
#Fig Supp 1G
df_ = pd.DataFrame({'actual':layer_to_predict[:,top10neuron_monkey], 'predicted':sorted_y_preds[:,top10neuron_monkey]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = '#007069'
plot_correlations(df_, color, r_position=(0.05,0.9), ax=ax, ev=evars[top10neuron_monkey])
ax.set_aspect('equal')
ax.axis('equal')
if save_figs is True: 
    plt.savefig(fig_dir + f'fig2_monkey_{monkey_name}_top10_corr.pdf', transparent=True,bbox_inches='tight', dpi=100 )
plt.show()

df = pd.DataFrame({'actual':layer_to_predict[:,med_neuron_monkey], 'predicted':sorted_y_preds[:,med_neuron_monkey]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = 'lightseagreen'
plot_correlations(df, color, r_position=(0.05,0.9), ax=ax, ev=evars[med_neuron_monkey])
ax.set_aspect('equal')
ax.axis('equal')
ax.set_ylabel(None)
if save_figs is True:
    plt.savefig(fig_dir + f'fig2_monkey_{monkey_name}_med_corr.pdf',transparent=True,bbox_inches='tight' )
plt.show()

df = pd.DataFrame({'actual':layer_to_predict[:,bottom10neuron_monkey], 'predicted':sorted_y_preds[:,bottom10neuron_monkey]})
fig, ax = plt.subplots(figsize=(0.95,0.95))
color = '#73BFB7'
plot_correlations(df, color, r_position=(0.05,0.9), ax=ax, ev=evars[bottom10neuron_monkey])
ax.set_aspect('equal')
ax.axis('equal')
ax.set_ylabel(None)
if save_figs is True:
    plt.savefig(fig_dir + f'fig2_monkey_{monkey_name}_bottom10_corr.pdf',transparent=True,bbox_inches='tight' )
plt.show()

In [ ]:
#Fig Supp 1I

med_evar = evars[med_neuron_monkey]
top_evar = evars[top10neuron_monkey]
bottom_evar = evars[bottom10neuron_monkey]


fig, ax = plt.subplots(figsize=(1.3,1.3))
sns.histplot(df_monkey[(df_monkey['Area']=='V4')&(df_monkey['Split-half r']>reli_theshold)&(df_monkey['SNR']>2)]['EV'], 
                color='#222222', ax=ax, binwidth=0.025, linewidth=0.4)

ax.axvline(top_evar, color='#007069', linestyle='--', linewidth=1, label='site #1', alpha=0.7)
ax.axvline(med_evar, color='lightseagreen', linestyle='--', linewidth=1, label='site #2', alpha=0.7)
ax.axvline(bottom_evar, color='#73BFB7', linestyle='--', linewidth=1, label='site #3', alpha=0.7)


data = df_monkey_all[(df_monkey_all['Split-half r']>reli_theshold)&(df_monkey_all['SNR']>2)&(df_monkey_all.control_shuffle==True)&(df_monkey_all.Area=='V4')]['EV']
per_25 = np.percentile(data.values, 25)
per_75 = np.percentile(data.values, 75)
ax.axvspan(per_25, per_75, alpha=0.3, color='blue', label='shuffle\ncontrol\nIQR',
        linewidth=0,
        )

ax.set_xlabel('EV fraction', fontsize=6, labelpad=1)
ax.set_ylabel('No. of V4 neural sites', fontsize=6, labelpad=1)

# Increase the tick label size for both x and y axes
ax.tick_params(axis='x', labelsize=6, width=0.5, length=2, pad=1)
ax.tick_params(axis='y', labelsize=6, width=0.5, length=2, pad=1)
legend = ax.legend(loc=(0.675,0.5), fontsize=6*0.8)
legend.get_frame().set_linewidth(0.2)
ax.legend_.remove()
ax.set_xticks(ticks=[0,0.1,0.2])
ax.spines[:].set_linewidth(0.5)
sns.despine()
if save_figs is True:
        plt.savefig(fig_dir + f'fig2_monkey_{monkey_name}_hist.pdf',transparent=True,bbox_inches='tight' )
plt.show()

## all monkeys together

In [ ]:
# Fig Supp 1K
fig, axes = plt.subplots(figsize=(3.2,1.3), ncols=3)

for a, (ax, (monkey_name, df_monkey)) in enumerate(zip(axes.flatten(), all_df_V1.groupby('Monkey Name'))):
	if monkey_name =='D':
		reli_theshold = 0.6
	else:
		reli_theshold = 0.8
	sns.histplot(df_monkey[(df_monkey['Area']=='V1')&(df_monkey['Split-half r']>reli_theshold)&(df_monkey['SNR']>2)]['EV'], 
		color='#222222', ax=ax,linewidth=0.4)
	if monkey_name=='D':
		# ax.set_title(f'Monkey {monkey_name[0].upper()}', fontsize=6*1.1)
		reli_theshold = 0.8
	else:
		reli_theshold = 0.8
		# ax.set_title(f'Monkey {monkey_name[0].upper()}', fontsize=6*1.1)
	data = df_monkeys_all[(df_monkeys_all['Monkey Name']==monkey_name)&(df_monkeys_all['Split-half r']>reli_theshold)&(df_monkeys_all['SNR']>2)&(df_monkeys_all.control_shuffle==True)&(df_monkeys_all.Area=='V1')]['EV']
	per_25 = np.percentile(data.values, 25)
	per_75 = np.percentile(data.values, 75)
	ax.axvspan(per_25, per_75, alpha=0.3, color='blue', label='shuffle\ncontrol\nIQR',
			linewidth=0,
	)

	mean_EV = df_monkey[(df_monkey['Area']=='V1')&(df_monkey['Split-half r']>0.6)&(df_monkey['SNR']>2)]['EV'].mean()
	ax.axvline(mean_EV, color='#EDAEAE', linestyle='--', linewidth=1, label=f'mean', alpha=0.7)


	ax.set_xlabel('EV fraction', fontsize=6, labelpad=1)
	ax.set_ylabel('No. of V1 neural sites', fontsize=6, labelpad=1)

	# Increase the tick label size for both x and y axes
	ax.tick_params(axis='x', labelsize=6, width=0.5, length=2, pad=1)
	ax.tick_params(axis='y', labelsize=6, width=0.5, length=2, pad=1)
	legend = ax.legend(loc=(0.38,0.6), fontsize=6*0.8)
	legend.get_frame().set_linewidth(0.2)
	ax.legend_.remove()

	sns.despine()
	ax.spines[:].set_linewidth(0.5)
	if a !=0:
		ax.set_ylabel(None)
plt.subplots_adjust(wspace=0.3)
if save_figs is True:
	plt.savefig(fig_dir + f'fig2_supp_monkey_all_V1_hist.pdf',transparent=True,bbox_inches='tight' )

In [ ]:
df_monkeys_all_v = pd.DataFrame()
for m, (monkey_name, df_monkey) in enumerate(df_monkeys_all.groupby('Monkey Name')):
	if monkey_name in ['Monkey $L_{A}$', 'Monkey $L_{D}$']:
		continue
	if monkey_name=='D':
		reli_theshold = 0.6
	else:
		reli_theshold = 0.8
	df_monkey.loc[:,'Visually Responsive']=[True if x > reli_theshold and y > 2 else False for x, y in zip(df_monkey['Split-half r'], df_monkey['SNR'])]
	fil_df_monkey = df_monkey[df_monkey['control_shuffle']==False]
	df_monkeys_all_v = pd.concat([df_monkeys_all_v, fil_df_monkey])

In [ ]:
## Fig Supp 1J

fig, ax = plt.subplots(figsize=(1.3,1.3))
sns.histplot(df_monkeys_all_v, x='EV', hue='Visually Responsive', fill=False, color='#222222', ax=ax,hue_order=[True,False],
        linewidth=0.4, bins=20, palette=['purple','gray'], stat='percent')
ax.axvline(df_monkeys_all_v[df_monkeys_all_v['Visually Responsive']==True]['EV'].mean(), color='purple', linestyle='--', linewidth=1, label='responsive\nmean EV', alpha=1)
ax.axvline(df_monkeys_all_v[df_monkeys_all_v['Visually Responsive']==False]['EV'].mean(), color='gray', linestyle='--', linewidth=1, label='non-responsive\nmean EV', alpha=1)

ax.set_xlabel('EV fraction', fontsize=6, labelpad=1)
ax.set_ylabel(f'Percent of Sites', fontsize=6, labelpad=1)
# Increase the tick label size for both x and y axes
ax.tick_params(axis='x', labelsize=6, width=0.5, length=2, pad=1)
ax.tick_params(axis='y', labelsize=6, width=0.5, length=2, pad=1)

legend = ax.get_legend()
handles1= legend.legend_handles
handles2, labels = ax.get_legend_handles_labels()
handles = handles1 + handles2
ax.legend(handles=handles, 
        labels=['responsive','non-responsive', 'responsive\nmean','non-responsive\nmean'], 
        fontsize=6*0.8,
        loc=(0.55,0.5),
        alignment='center')
ax.set_xticks(ticks=np.arange(0,0.9,0.2))
ax.legend_.get_frame().set_linewidth(0.2)
ax.legend_.remove()
ax.spines[:].set_linewidth(0.45)
sns.despine()
if save_figs is True:
        plt.savefig(fig_dir + f'fig2_supp_monkey_all_visual_nonvisual.pdf',transparent=True,bbox_inches='tight' )
plt.show()

In [ ]:
# what percentage of sites are visually responsive across all monkeys?
total_neurons = len(df_monkeys_all_v)
total_visually_responsive_neurons = len(df_monkeys_all_v[df_monkeys_all_v['Visually Responsive']==True])
percentage_visually_responsive = (total_visually_responsive_neurons / total_neurons) * 100
print(f'Total neurons across all monkeys: {total_neurons}')
print(f'Total visually responsive neurons across all monkeys: {total_visually_responsive_neurons}')
print(f'Percentage of visually responsive neurons across all monkeys: {percentage_visually_responsive:.2f}%')

In [ ]:
# what percentage of sites are visually responsive across all monkeys?
total_neurons_V4 = len(df_monkeys_all_v[df_monkeys_all_v['Area']=='V4'])
total_visually_responsive_neurons_V4 = len(df_monkeys_all_v[(df_monkeys_all_v['Visually Responsive']==True)&(df_monkeys_all_v['Area']=='V4')])
percentage_visually_responsive_V4 = (total_visually_responsive_neurons_V4 / total_neurons_V4) * 100
print(f'Total V4 neurons across all monkeys: {total_neurons_V4}')
print(f'Total visually responsive V4 neurons across all monkeys: {total_visually_responsive_neurons_V4}')
print(f'Percentage of visually responsive V4 neurons across all monkeys: {percentage_visually_responsive_V4:.2f}%')

In [ ]:
## look at sites in V1 that are visually responsive across all monkeys
total_neurons_V1 = len(df_monkeys_all_v[df_monkeys_all_v['Area']=='V1'])
total_visually_responsive_neurons_V1 = len(df_monkeys_all_v[(df_monkeys_all_v['Visually Responsive']==True)&(df_monkeys_all_v['Area']=='V1')])
percentage_visually_responsive_V1 = (total_visually_responsive_neurons_V1 / total_neurons_V1) * 100
print(f'Total V1 neurons across all monkeys: {total_neurons_V1}')
print(f'Total visually responsive V1 neurons across all monkeys: {total_visually_responsive_neurons_V1}')
print(f'Percentage of visually responsive V1 neurons across all monkeys: {percentage_visually_responsive_V1:.2f}%')

##### calculate table totals

In [ ]:
import json
with open(os.path.join(project_root, 'results/fig_2', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
    
main_monkey_name = 'L'
subsample_monkey_name = 'A'

df_monkey_L_A = pd.DataFrame()

for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_2/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	subsampled_monkey_df = make_monkey_df(subsampled_monkey_stats, dataset_types=['SNR'])
	subsampled_monkey_df['Subsample Seed'] = seed
	df_monkey_L_A = pd.concat([df_monkey_L_A, subsampled_monkey_df])


subsample_monkey_name = 'D'
df_monkey_L_D = pd.DataFrame()

for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_2/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	subsampled_monkey_df = make_monkey_df(subsampled_monkey_stats, dataset_types=['SNR'])
	subsampled_monkey_df['Subsample Seed'] = seed
	df_monkey_L_D = pd.concat([df_monkey_L_D, subsampled_monkey_df])


In [ ]:
df_monkey_L_A['Monkey Name'] = r'Monkey $L_{A}$'
df_monkey_L_D['Monkey Name'] = r'Monkey $L_{D}$'
df_monkeys_all = pd.concat([df_monkeys_all,df_monkey_L_A, df_monkey_L_D, ])

In [ ]:
df_monkeys_filtered = pd.DataFrame()
reli_theshold = 0.8
for monkey_name in df_monkeys_all['Monkey Name'].unique():
	df_monkey_filt = df_monkeys_all[(df_monkeys_all['Monkey Name']==monkey_name)&(df_monkeys_all['control_shuffle']==False)].copy()
	df_monkeys_filtered = pd.concat([df_monkeys_filtered, df_monkey_filt])

dir_map = {'V4':'V1→V4', 'V1':'V4→V1'}
df_monkeys_filtered['Direction'] = df_monkeys_filtered['Area'].map(dir_map)

## 

df_monkeys_filtered.loc[:,'Visually Responsive']=False
df_monkeys_filtered.loc[:,'SNR > 2']=[True if y > 2 else False for _, y in zip(df_monkeys_filtered['Split-half r'], df_monkeys_filtered['SNR'])]
## loop through each monkey and calculate reliable based on reli_theshold. reli_threshold will be different for monkey D and r'Monkey $L_{D}$'
df_monkeys_filtered['reliable'] = False
for monkey_name in df_monkeys_filtered['Monkey Name'].unique():
	if monkey_name in [r'Monkey $L_{D}$', 'D']:
		threshold = 0.6
		print(f'Setting reliability threshold for {monkey_name} to {threshold}')
	else:
		threshold = 0.8
	df_monkeys_filtered.loc[(df_monkeys_filtered['Monkey Name']==monkey_name)&(df_monkeys_filtered['Split-half r']>threshold), 'reliable'] = True
	df_monkeys_filtered.loc[(df_monkeys_filtered['Monkey Name']==monkey_name)&(df_monkeys_filtered['Split-half r']>threshold)&(df_monkeys_filtered['SNR']>2), 'Visually Responsive'] = True

df_all = df_monkeys_filtered.copy().reset_index()
df_all['cell type']='all'
df_snr = df_monkeys_filtered[df_monkeys_filtered['SNR > 2']==True].reset_index()
df_snr['cell type']='SNR > 2'
df_filt = df_monkeys_filtered[df_monkeys_filtered['Visually Responsive']==True].reset_index()
df_filt['cell type']=f'SNR > 2 &\nsplit-half r >{reli_theshold}'
df_reli = df_monkeys_filtered[df_monkeys_filtered['reliable']==True].reset_index()
df_reli['cell type']=f'split-half r >{reli_theshold}'
df_conditions_comparisons_monkey = pd.concat([df_all,df_reli,df_snr, df_filt]).reset_index()


In [ ]:
# Fig Supp 1L

all_stars = get_ext_fig_stars(df_conditions_comparisons_monkey,x_label='Monkey Name', stat_names = [f'all_split-half r >{reli_theshold}', f'split-half r >{reli_theshold}_SNR > 2', f'SNR > 2_SNR > 2 &\nsplit-half r >{reli_theshold}'])
fig, ax = plt.subplots(figsize=(4,2))

xtick_dict = {'L':r'Monkey $\text{L}$', 'A':r'Monkey $\text{A}$', 'D':r'Monkey $\text{D}$',
             r'Monkey $L_{A}$':r'Monkey $\text{L}_{\text{A}}$', r'Monkey $L_{D}$':r'Monkey $\text{L}_{\text{D}}$',}
violin=sns.violinplot(df_conditions_comparisons_monkey,  y='EV', x='Monkey Name', hue='cell type', ax=ax,
            palette=['lightgray','#ffb3ba','#bae1ff','#d0a0ec'],
            inner_kws={'box_width':2, 'marker':'_', 'markersize':3,
                            'markeredgewidth':1,'alpha':0.5},
            linewidth=0.4,cut=0)
ax.set_xlabel(None)
ax.set_ylabel(f'EV fraction', fontsize=6, labelpad=1)
ax.set_xticks(ticks=[0,1,2,3,4], labels=[xtick_dict[m] for m in df_conditions_comparisons_monkey['Monkey Name'].unique()])
ax.set_yticks(ticks=[0,0.2,0.4,0.6, 0.8])
ax.tick_params(axis='x', labelsize=6, width=0.5, length=2, pad=0)
ax.tick_params(axis='y', labelsize=6, width=0.5, length=2, pad=1)
ax.legend(loc=(0.37,0.55), fontsize=5)
ax.legend_.get_frame().set_linewidth(0.2)

ax.spines[:].set_linewidth(0.5)
sns.despine()

x_positions = get_x_positions(violin)   
y_positions = [0.7,0.7,0.7,
            0.6,0.6,0.6,
            0.35,0.35,0.35,
            0.6,0.6,0.6,
            0.6,0.6,0.6
            ]
for count, (x_pos, star, y_pos) in enumerate(zip(x_positions, all_stars,y_positions)):
    ax.text(x=x_pos-0.17, y=y_pos, s=star, 
            fontsize=5)
ax.legend_.get_frame().set_linewidth(0.5)
ax.legend_.remove()
plt.savefig(fig_dir + f'fig2_supp_all_monkeys_EV_filter_comparisons.pdf',transparent=True,bbox_inches='tight' )
plt.show()

In [ ]:
df_conditions_comparisons_monkey['cell type'].unique()

In [ ]:
new_legends = ['all', 'split-half r > ' + r'$thr.$', 'SNR > 2',
       'SNR > 2 &\nsplit-half r > ' + r'$thr.$']

In [ ]:
# Fig Supp 1L

fig, ax = plt.subplots(figsize=(4,2))

xtick_dict = {'L':r'Monkey $\text{L}$', 'A':r'Monkey $\text{A}$', 'D':r'Monkey $\text{D}$',
             r'Monkey $L_{A}$':r'Monkey $\text{L}_{\text{A}}$', r'Monkey $L_{D}$':r'Monkey $\text{L}_{\text{D}}$',}
violin=sns.violinplot(df_conditions_comparisons_monkey,  y='EV', x='Monkey Name', hue='cell type', ax=ax,
            palette=['lightgray','#ffb3ba','#bae1ff','#d0a0ec'],
            inner_kws={'box_width':2, 'marker':'_', 'markersize':3,
                            'markeredgewidth':1,'alpha':0.5},
            linewidth=0.4,cut=0)
ax.set_xlabel(None)
ax.set_ylabel(f'EV fraction', fontsize=6, labelpad=1)
ax.set_xticks(ticks=[0,1,2,3,4], labels=[xtick_dict[m] for m in df_conditions_comparisons_monkey['Monkey Name'].unique()])
ax.set_yticks(ticks=[0,0.2,0.4,0.6, 0.8])
ax.tick_params(axis='x', labelsize=6, width=0.5, length=2, pad=0)
ax.tick_params(axis='y', labelsize=6, width=0.5, length=2, pad=1)
handles, old_labels = ax.get_legend_handles_labels()
ax.legend(loc=(0.365,0.69), fontsize=5, labels=new_legends, handles=handles)
ax.legend_.get_frame().set_linewidth(0.2)

ax.spines[:].set_linewidth(0.5)
sns.despine()

x_positions = get_x_positions(violin)   
y_positions = [0.7,0.7,0.7,
            0.6,0.6,0.6,
            0.35,0.35,0.35,
            0.6,0.6,0.6,
            0.6,0.6,0.6
            ]
for count, (x_pos, star, y_pos) in enumerate(zip(x_positions, all_stars,y_positions)):
    ax.text(x=x_pos-0.17, y=y_pos, s=star, 
            fontsize=5)
ax.legend_.get_frame().set_linewidth(0.5)
# ax.legend_.remove()
plt.savefig(fig_dir + f'fig2_supp_all_monkeys_EV_filter_comparisons.pdf',transparent=True,bbox_inches='tight' )
plt.show()

## GLM poisson comparison

In [ ]:
palette_dict = {}
palette_dict['V4'] = {'Ridge':'#72BEB7', 'Gamma GLM':'#136a66'}
palette_dict['V1'] = {'Ridge':'#EDAEAE', 'Gamma GLM':'#a85959'}
palette_dict['L23'] = {'Ridge':'#72BEB7', 'Gamma GLM':'#136a66'}
palette_dict['L4'] = {'Ridge':'#EDAEAE', 'Gamma GLM':'#a85959'}

palette_dict['V4'] = {'Ridge':'#72BEB7', 'Poisson GLM':'#136a66'}
palette_dict['V1'] = {'Ridge':'#EDAEAE', 'Poisson GLM':'#a85959'}
palette_dict['L23'] = {'Ridge':'#72BEB7', 'Poisson GLM':'#136a66'}
palette_dict['L4'] = {'Ridge':'#EDAEAE', 'Poisson GLM':'#a85959'}


dataset_type_dict = {'A': ['SNR','RF_thin', 'RF_large'],
                    'L': ['SNR','RF_thin', 'RF_large'],
                    'D': ['SNR']}

In [ ]:
monkey_names = ['L','A', 'D']
df_monkeys_all = pd.DataFrame()
for monkey_name in monkey_names:
	reli_theshold = 0.8 if monkey_name in ['L','A'] else 0.6
	monkey_stats = all_monkey_stats[monkey_name]
	df_monkey_all = make_monkey_df_poisson(monkey_stats, dataset_types=dataset_type_dict[monkey_name], add_glm_ev = True)
	df_monkey = df_monkey_all[(df_monkey_all['Split-half r']>=reli_theshold)&(df_monkey_all['control_shuffle']==False)].reset_index()
	df_monkey.loc[:,'Monkey Name'] = monkey_name.capitalize()
	df_monkeys_all = pd.concat([df_monkeys_all, df_monkey], ignore_index=True)
df_monkeys_all_melt = pd.melt(df_monkeys_all, id_vars=['Dataset Type', 'Activity Type', 'Date', 'Area', 'control_shuffle', 'Cell Number', 'Monkey Name', 'SNR', 'Split-half r'], value_vars=['EV','EV_glm'], var_name='Model Type', value_name='EV Value')
df_monkeys_all_melt['Model Type'] = df_monkeys_all_melt['Model Type'].replace({'EV':'Ridge', 'EV_glm':'Poisson GLM'})


In [ ]:

monkey_label_dict = {'L':'Monkey L', 'A':'Monkey A', 'D':'Monkey D'}
figsize=(3.4,1.8)
fontsize=6


x='Monkey Name'
hue='Model Type'
y='EV Value'


hue_order = df_monkeys_all_melt['Model Type'].unique()
x_order = df_monkeys_all_melt['Monkey Name'].unique()
fig, axes = plt.subplots(figsize=figsize, sharey=True, nrows=1, ncols=2)
for area, ax in zip(['V4','V1'], axes.flatten()):
	mini_df = df_monkeys_all_melt[df_monkeys_all_melt['Area']==area]
	direction = 'V1→V4' if area=='V4' else 'V4→V1'
	sns.violinplot(data=mini_df, y=y, hue=hue, ax=ax, x=x,
				linewidth=0,cut=0,inner_kws={'box_width':2, 'whis_width':0.5,
										'marker':'_', 'markersize':3,
										'markeredgewidth':0.8,
										}, saturation=1,palette=palette_dict[area],
				hue_order=hue_order, order=x_order,
				)
	# compute y for stars a bit above the taller of the pair
	ymins, ymaxs = ax.get_ylim()
	yrange = ymaxs - ymins
	pvals = {}
	for x_i in x_order:
		pvals[x_i] = get_comparison_test_stars(mini_df[mini_df[x]==x_i], independent_variable=hue, neuron_property=y, print_pval=True,
												mouse_or_date='Date',perm_type='paired')
	for i, act in enumerate(x_order):
		if act not in pvals:
			continue
		stars = pvals[act]
		# find max y at this category from plotted data to place stars above
		sub = mini_df[mini_df[x]==act][y].to_numpy()
		sub = sub[np.isfinite(sub)]
		y_top = (np.percentile(sub, 99) if sub.size else ymaxs) + 0.03*yrange
		color = "#C0C0C0" if stars=="n.s." else "black"
		ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=fontsize, color=color)

	sns.despine()
	ax.set_xlabel('')
	ax.set_xticklabels(labels=[monkey_label_dict[l] for l in x_order], rotation=0, fontsize=fontsize)
	ax.tick_params(labelsize=fontsize, pad=1)
	ax.set_ylabel('Predictability (EV)', fontsize=fontsize, labelpad=1)
	ax.set_title(f'{direction}', fontsize=fontsize*1.2)
	if ax.legend_:
		ax.legend(title='', fontsize=fontsize*0.8, loc=(0.57,0.8))
		ax.legend_.get_frame().set_linewidth(0.3)
	# frame width
	ax.spines[:].set_linewidth(0.5)
plt.subplots_adjust(wspace=0.1, hspace=0.3, left=0.1, right=0.95, top=0.9, bottom=0.2)
if save_figs:
	fig.savefig(os.path.join(fig_dir,f'R1.1_fig2_monkey_interareal_predictability_gamma_glm_comparison_all_monkeys.pdf'), dpi=300, bbox_inches='tight', transparent=True)
plt.show()